# Download *z_cl* & *New Performance Report By Processing Location* for the following

In [1]:
import csv
import glob
import shutil
import os
import pandas as pd
import time, os, fnmatch, shutil
import openpyxl
import numpy as np

#Create variable timestamp 
t = time.localtime()
timestamp = time.strftime('%Y%m%d_%H%M', t)

# Read .csv from Downloads folder and get the reporting month
perform = pd.read_csv('C://Users//nnagaraj//Downloads//new-performance-report-by-processing-location.csv')

# append 01 to reporting_month column and convert it to datetime format
perform['reporting_month'] = pd.to_datetime(perform['reporting_month'].
                                           apply(lambda x: f"{x}01").
                                           astype(str), infer_datetime_format = True)

# String of reporting month
report_str_month = perform['reporting_month'][:1].dt.month_name(locale='English')[0]

# int of reporting month
report_month = str(perform['reporting_month'][:1].dt.month[0])

# int of reporting year
report_year = str(perform['reporting_month'][:1].dt.year[0])

# formatting directory name:S:\TANF Reports\!2021 TANF Reports
dir_name = 'S://TANF Reports' + "//" + '!' + report_year + " " + "TANF Reports"

# formatting folder name: 11-November 2021
folder_name = report_month + "-" + report_str_month + " " + report_year

# change directory
os.chdir(dir_name)

# create folder in directory
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Move all .csv files in Downloads folder to S:drive(destination) folder
path = 'C://Users//nnagaraj//Downloads'
os.chdir(path)
source = os.listdir(path)
destination = os.path.join(dir_name, folder_name )

for files in source:  
    if files.endswith(".csv"):
        shutil.move(files, destination)  
        

# Convert all .csv files in S:drive to Excel        
os.chdir(destination)        
for filename in os.listdir(destination):
    if filename.endswith(".csv"):
        df = pd.read_csv(os.path.join(destination, filename))
        x = filename.replace("-", " ") + ' - ' + report_str_month + ' ' + report_year.title()
        writer = pd.ExcelWriter(str(x).title().replace(".Csv", "")+".xlsx")
        df.to_excel(writer, sheet_name = report_str_month, index = False, header = filename)
        writer.save()
        continue
    else:
        continue       

# Now that all csv have been converted to excel, delete all the csv files in the directory        
files = [file for file in glob.glob("*.csv") if not file.endswith("0.csv")]
for file in files:
    os.remove(file)



In [2]:

# load caseload report
cl = pd.read_excel("Z_Cl -" + " " + report_str_month + " " + report_year + ".xlsx")

# create copy of cl
#df_cl = cl.copy()

# create subset by selecting required columns
cl_df = cl[['local_office', 'name.1','program_unit_id', 'permissions']]#, 'Program Type']]


# Create Program column based on +ve or -ve case number
cl_df['Program Type'] = cl_df['program_unit_id'].apply(lambda x: 'TEA' if x >= 0 else 'Work Pays')


# Rename column names
cl_dff = cl_df.rename(columns = {'local_office':'Local Office',
                                'name.1': 'Case Manager',
                                'program_unit_id': 'Case Number'})

# casenumbers, case managers by local office
cl_tab = cl_dff.drop(['permissions'], axis = 'columns')
cl_tab = cl_tab.sort_values(['Local Office'], ascending = [True])


# groupby lo, cm and program type and unique cn
cl_dff_uni = cl_dff.groupby(['Local Office', 'Case Manager', 'Program Type'], as_index = False)['Case Number'].nunique()

# groupby lo and unique casemanagers
cl_dff_uni = cl_dff.groupby(['Local Office', 'Case Manager', 'Program Type'], as_index = False)['Case Number'].nunique()
cl_dff_uni = cl_dff_uni.rename(columns = {'Case Number': 'Caseload'})

### unique case managers by local office
###cm_uni = cl_dff_uni.groupby(['Local Office'])['Case Manager'].nunique().to_frame('Number of WFS').reset_index()
# subset with permissions containing 'Specialist'
df_wfs_uni = cl_dff[cl_dff['permissions'].str.contains("Specialist")]
wfs_uni = df_wfs_uni[['Local Office', 'Case Manager', 'Case Number']]
cm_uni = wfs_uni.groupby(['Local Office'])['Case Manager'].nunique().to_frame('Number of WFS').reset_index()

# load performance report 
pr = pd.read_excel("New Performance Report By Processing Location -" + " " + report_str_month + " " + report_year + ".xlsx")


# subset for first 10 columns
pr_df = pr.iloc[ : ,1:12]

# Rename columns
pr_df = pr_df.rename(columns = {'short_description': 'Local Office','total_count': 'Total Caseload'})

pr_df['Caseload'] = pr_df['Total Caseload']-pr_df['TEA - Child Only']

                                                            
# Rearrange columns
##pr_df = pr_df[['Local Office','Caseload', 'Work Pays', 'TEA','Exempt',
               ##'Deferred','Prog. Sanc','Assigned Universe','%Progress.Sanction']]#,'% Deferred' ]]
    
pr_df = pr_df.drop(['Prog. Sanc'], axis = 'columns')
    
    
pr_df = pr_df[['Local Office', 'Caseload', 'TEA - Single Parent',
       'TEA - Two Parent', 'Work Pays - Single Parent',
       'Work Pays - Two Parent', 'Exempt', 'Assigned Universe', '%Progress.Sanction']]

# fill nan with 0
pr_dff = pr_df.fillna(0)

# merge number of wfs column to pr_dff on local office
aggregate = pd.merge(pr_dff,cm_uni, on='Local Office')

# insert average caseload column
aggregate['Avg. Caseload'] = ((aggregate['Caseload']/aggregate['Number of WFS'])).round()

# Stats
# Creating variables for dataframe

max_caseload = aggregate.loc[(aggregate['Caseload'].idxmax())].to_frame().T.reset_index()
min_caseload = aggregate.loc[(aggregate['Caseload'].idxmin())].to_frame().T.reset_index()
max_caseload_wfs = aggregate.loc[(aggregate['Number of WFS'].idxmax())].to_frame().T.reset_index()
min_caseload_wfs= aggregate.loc[(aggregate['Number of WFS'].idxmin())].to_frame().T.reset_index()
max_per_san = aggregate.loc[(aggregate['%Progress.Sanction'].idxmax())].to_frame().T.reset_index()
min_per_san = aggregate.loc[(aggregate['%Progress.Sanction'].idxmin())].to_frame().T.reset_index()

# Local Office Column
lo_max_caseload = max_caseload.loc[0,'Local Office']
lo_min_caseload = min_caseload.loc[0,'Local Office']
lo_max_caseload_wfs = max_caseload_wfs.loc[0,'Local Office']
lo_min_caseload_wfs = min_caseload_wfs.loc[0,'Local Office']
lo_max_per_san = max_per_san.loc[0,'Local Office']
lo_min_per_san = min_per_san.loc[0,'Local Office']

# Number column
num_max_caseload = max_caseload.loc[0,'Caseload']
num_min_caseload = min_caseload.loc[0,'Caseload']
num_max_caseload_wfs = max_caseload_wfs.loc[0,'Avg. Caseload']
num_min_caseload_wfs = min_caseload_wfs.loc[0,'Avg. Caseload']
num_max_per_san = max_per_san.loc[0,'%Progress.Sanction']
num_min_per_san = min_per_san.loc[0,'%Progress.Sanction']

# DataFrame
stat_frame = pd.DataFrame({'Summary_Matrix': ['LO with Maximum Caseload',
                                              'LO with Minimum Caseload',
                                             'LO with Maximum Caseload, WFS Ratio',
                                             'LO with Minimum Caseload, WFS Ratio',
                                             'LO with Maximum Sanction Percentage',
                                             'LO with Minimum Sanction Percentage'],
                           'Local_Office':[lo_max_caseload,
                                           lo_min_caseload,
                                           lo_max_caseload_wfs,
                                           lo_min_caseload_wfs,
                                           lo_max_per_san,
                                           lo_min_per_san],
                           'Number': [num_max_caseload,
                                      num_min_caseload,
                                      num_max_caseload_wfs,
                                      num_min_caseload_wfs,
                                      num_max_per_san,
                                      num_min_per_san]},
                         columns = ['Summary_Matrix','Local_Office','Number'])

##### EXCEL TAB ######
#pr_dff 
#cl_dff 
#cl_dff_uni
#cl_dff_uni
#aggregate
#stat_frame

# Save statistics in a new tab of Caseload Breakdown By Local Office - August 2019.xlsx 
#file_name = 'jaiganesh' + ' - ' + str_prev_month + ' ' + now.strftime('%Y') + '.xlsx'
file_name = 'Caseload Breakdown by Local Office' + ' - ' + report_str_month + ' ' + report_year + '.xlsx'
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')



# Save dataframe in a new tab of Caseload Breakdown By Local Office - August 2019.xlsx    
#df_sanction_count.to_excel(writer, sheet_name= report_str_month + '_sanction', index=False)
cl_tab.to_excel(writer, sheet_name= report_str_month + '_caseload', index=False)
cl_dff_uni.to_excel(writer, sheet_name= report_str_month + '_casemanager', index=False)
aggregate.to_excel(writer, sheet_name= report_str_month + '_aggregate', index=False)
stat_frame.to_excel(writer, sheet_name= report_str_month + '_stats', index=False)
#caseload_count.to_excel(writer, sheet_name= report_str_month + '_caseload_count_cn' , index=False)
writer.save()
writer.close()


C:\Users\nnagaraj\AppData\Local\Temp/ipykernel_7124/3877618970.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cl_df['Program Type'] = cl_df['program_unit_id'].apply(lambda x: 'TEA' if x >= 0 else 'Work Pays')
